# Not Using Pauli Frame Update

In [1141]:
from qiskit import __version__
print(__version__)

1.4.2


In [1142]:
from qiskit import QuantumCircuit, ClassicalRegister
from qiskit.quantum_info import Statevector, state_fidelity, partial_trace, DensityMatrix, Pauli
from qiskit.quantum_info.operators import Operator
from qiskit_aer import AerSimulator
from qiskit.visualization import plot_histogram
from qiskit import transpile 
import numpy as np
from qiskit_aer.noise import NoiseModel, depolarizing_error, ReadoutError
from qiskit.circuit.controlflow import IfElseOp
from qiskit.circuit.library import XGate, ZGate
import matplotlib.pyplot as plt

In [1143]:
import importlib
import steane_ec_decoder2
importlib.reload(steane_ec_decoder2)
from steane_ec_decoder2 import lookup

# Function for Encoding

In [1144]:
def encoding(qc: QuantumCircuit):
    theta = np.arctan(np.sqrt((np.sqrt(5) - 1) / 2))
    amp_0 = np.cos(theta/2)
    amp_1 = np.sin(theta/2)
    qc.initialize([amp_0, amp_1], 0)
    for i in range(7):
        qc.id(i)
    for i in range(4, 7):
        qc.h(i)
    qc.cx(0, 1)
    qc.cx(0, 2)
    qc.cx(6, 0)
    qc.cx(6, 1)
    qc.cx(5, 0)
    qc.cx(6, 3)
    qc.cx(5, 2)
    qc.cx(4, 1)
    qc.cx(5, 3)
    qc.cx(4, 2)
    qc.cx(4, 3)

# Functions for Logical |0> and Logical |+>

In [1145]:
# — Custom Hadamard —
h_sub = QuantumCircuit(1, name='H_custom')
h_sub.h(0)
H_custom = h_sub.to_gate()

# — Custom CNOT —
cx_sub = QuantumCircuit(2, name='CX_custom')
cx_sub.cx(0, 1)
CX_custom = cx_sub.to_gate()

In [1146]:
def logical_zero(qc: QuantumCircuit, first_qubit: int):
    for i in [0, 1, 3]:
        qc.append(H_custom, [first_qubit+i])
    
    qc.append(CX_custom, [first_qubit, first_qubit+4])
    qc.append(CX_custom, [first_qubit+1, first_qubit+2])
    qc.append(CX_custom, [first_qubit+3, first_qubit+5])
    qc.append(CX_custom, [first_qubit, first_qubit+6])
    qc.append(CX_custom, [first_qubit+3, first_qubit+4])
    qc.append(CX_custom, [first_qubit+1, first_qubit+5])
    qc.append(CX_custom, [first_qubit, first_qubit+2])
    qc.append(CX_custom, [first_qubit+5, first_qubit+6])

In [1147]:
def logical_plus(qc: QuantumCircuit, first_qubit: int):
    logical_zero(qc, first_qubit)
    for i in range(7):
        qc.append(H_custom, [first_qubit+i])

# Functions for Stabilizer Extraction 

In [1148]:
def unflag(qc: QuantumCircuit, c1: ClassicalRegister, c2: ClassicalRegister):
    logical_plus(qc, 7)
    
    for i in range(7):
        qc.cx(i, i+7)
    qc.measure([7,8,9,10,11,12,13], c1)
    
    qc.reset([7,8,9,10,11,12,13])
    
    logical_zero(qc, 7)
    
    for i in range(7):
        qc.cx(i+7, i)
        qc.h(i+7)
    qc.measure([7,8,9,10,11,12,13], c2)

In [1149]:
def flag(qc: QuantumCircuit, first_qubit: int, c: ClassicalRegister):
    for i in [first_qubit+7, first_qubit+11, first_qubit+12]:
        qc.h(i)
        
    qc.cx(first_qubit+7, first_qubit+4)
    qc.cx(first_qubit+6, first_qubit+8)
    qc.cx(first_qubit+5, first_qubit+9)
    qc.cx(first_qubit+7, first_qubit+9)
    qc.cx(first_qubit+7, first_qubit)
    qc.cx(first_qubit+4, first_qubit+8)
    qc.cx(first_qubit+1, first_qubit+9)
    qc.cx(first_qubit+7, first_qubit+2)
    qc.cx(first_qubit+3, first_qubit+8)
    qc.cx(first_qubit+6, first_qubit+9)
    qc.cx(first_qubit+7, first_qubit+8)
    qc.cx(first_qubit+7, first_qubit+6)
    qc.cx(first_qubit+5, first_qubit+8)
    qc.cx(first_qubit+2, first_qubit+9)
    
    qc.cx(first_qubit+4, first_qubit+10)
    qc.cx(first_qubit+11, first_qubit+6)
    qc.cx(first_qubit+12, first_qubit+5)
    qc.cx(first_qubit+12, first_qubit+10)
    qc.cx(first_qubit, first_qubit+10)
    qc.cx(first_qubit+11, first_qubit+4)
    qc.cx(first_qubit+12, first_qubit+1)
    qc.cx(first_qubit+2, first_qubit+10)
    qc.cx(first_qubit+11, first_qubit+3)
    qc.cx(first_qubit+12, first_qubit+6)
    qc.cx(first_qubit+11, first_qubit+10)
    qc.cx(first_qubit+6, first_qubit+10)
    qc.cx(first_qubit+11, first_qubit+5)
    qc.cx(first_qubit+12, first_qubit+2)
    
    for i in [first_qubit+7, first_qubit+11, first_qubit+12]:
        qc.h(i)
    
    qc.measure([first_qubit+7, first_qubit+8, first_qubit+9, first_qubit+10, first_qubit+11, first_qubit+12], c)

# Function for QEC

In [1150]:
def QEC(qc: QuantumCircuit, c: ClassicalRegister, c1: ClassicalRegister, c2: ClassicalRegister):
    flag(qc, 0, c)
    qc.reset([7,8,9,10,11,12])
    
    # Path 1: if c is all 0, then do nothing
    path1 = QuantumCircuit(14)
    path1.add_register(c, c1, c2)
    # Path 2: if c is anything else
    path2 = QuantumCircuit(14)
    path2.add_register(c, c1, c2)
    unflag(path2, c1, c2)
    
    first_if = IfElseOp((c, 0), true_body=path1, false_body=path2)
    qc.append(first_if, list(range(14)), c[:] + c1[:] + c2[:])

# Encoding + 1 Round of EC

In [1151]:
noise_model = NoiseModel()

noise_model.add_all_qubit_quantum_error(depolarizing_error(0.005,1), ['id'])
noise_model.add_all_qubit_quantum_error(depolarizing_error(0.005,1), ['h'])
noise_model.add_all_qubit_quantum_error(depolarizing_error(0.05,2), ['cx'])

# 5% chance of flipping 0 <-> 1
readout_err = ReadoutError([[0.995, 0.005],  # P(measured 0 | actual 0), P(1 | 0)
                            [0.005, 0.995]]) # P(0 | 1), P(1 | 1)

# Apply to all qubits being measured
noise_model.add_readout_error(readout_err, [7]) 
noise_model.add_readout_error(readout_err, [8])
noise_model.add_readout_error(readout_err, [9])
noise_model.add_readout_error(readout_err, [10])
noise_model.add_readout_error(readout_err, [11])
noise_model.add_readout_error(readout_err, [12])
noise_model.add_readout_error(readout_err, [13])


In [1152]:
qc = QuantumCircuit(14)

c = ClassicalRegister(6, "c")
c1 = ClassicalRegister(7, "c1")
c2 = ClassicalRegister(7, "c2")

qc.add_register(c, c1, c2)

encoding(qc)
QEC(qc, c, c1, c2)

qc.save_statevector(label='statevector_post', pershot=True, conditional=True)

backend = AerSimulator(noise_model=noise_model)
transpiled = transpile(qc, backend, optimization_level=0)
job = backend.run(transpiled, shots=100, memory=True)
result = job.result()
memory = result.get_memory()

#print(result)
print(memory)

['1100111 0111000 000011', '0011111 1111011 001100', '0000000 0000000 000000', '0000000 0000000 000000', '0001110 1000110 100000', '0000000 0000000 000000', '1010010 1101100 001001', '0100000 1000000 110010', '1100110 0001110 001000', '1101110 0110010 011010', '0000000 0000000 000000', '0000000 0000000 000000', '0000000 0000000 000000', '0101010 1111111 011000', '1110111 0111000 001000', '1010110 1110010 110010', '1101110 1100100 011101', '1001100 1010110 001100', '0100000 1101001 110101', '0000000 0000000 000000', '1100111 0010100 110000', '0100111 0101010 101010', '0000000 0000000 000000', '1101110 1110101 011011', '1111100 0101010 111010', '1000001 1001111 110010', '0101011 0000001 001000', '1111100 0001111 001000', '0011000 1100111 111100', '1111110 0011100 000111', '0011111 0111011 100110', '0000111 0101101 000101', '0101010 0000011 000010', '0000000 0000000 000000', '1111110 1010100 001001', '1000000 0101110 100000', '0101000 1100101 001100', '1100110 1000000 101001', '1101100 10

In [1153]:
from collections import defaultdict

In [1154]:
hex_to_sv = []
hex_mem = result.data()['memory']

seen = defaultdict(int)
for mem in hex_mem:
    idx = seen[mem]
    val = result.data()['statevector_post'][mem][idx]
    hex_to_sv.append({mem:val})
    seen[mem] += 1
    
#print(hex_to_sv)

In [1155]:
sv_correction = []
for i, result in enumerate(hex_to_sv):
    str13, sv13 = next(iter(result.items()))
    cx, cz = lookup(memory[i])
    cz = cz + "I"*7
    cx = cx + "I"*7
    Pauli_cz = Pauli(cz)
    Pauli_cx = Pauli(cx)
    U_cz = Operator(Pauli_cz)
    U_cx = Operator(Pauli_cx)
    sv13 = sv13.evolve(U_cz)
    sv13 = sv13.evolve(U_cx)
    sv_correction.append(sv13)
#print(sv_correction)

In [1156]:
test = QuantumCircuit(7)
encoding(test)
test = DensityMatrix(test)

sum = 0
for sv in sv_correction:
    sv = partial_trace(sv, [7, 8, 9, 10, 11, 12, 13])
    if (np.round(state_fidelity(sv, test)) == 1.0):
        sum += 1

In [1157]:
print(sum)

21
